In [ ]:
import os
os.environ['KERAS_BACKEND'] = 'torch'
import keras
from utils import create_mask_dataset
from layers import UnetBackbone
from utils import create_mask_dataset, bce_dice_loss, iou_coef, dice_coef

In [ ]:
val_ds = create_mask_dataset(
    img_folder="bubble_masks/val/images",
    mask_folder="bubble_masks/val/masks",
    input_shape=(960, 1920, 1),
).batch(8)

In [ ]:
model = keras.models.load_model('unet_ss_best.keras', compile=False)

model.compile(
    loss=bce_dice_loss,
    metrics=[iou_coef, dice_coef],
)

In [ ]:
import matplotlib.pyplot as plt
results = model.evaluate(val_ds)

In [ ]:
results